# Detecting the fiducial particles in EM images

Fiducial particles/markers are used in correlated light and electron microscopy (CLEM) to enable accurate overlaying of fluorescence (LM) and electron microscopy (EM) images. The fiducial particles in EM images appear as bright circular regions with dark central spot. 

In this notebook, we **detect fiducial particles** in **EM images** using the template matching algorithm. As a template we use an artificially generated bright image with dark spot in the middle that resembles with its appearence the central part of the fiducial particle. After matching this template to the individual fiducial particles we detect fiducial clusters that consists of at least three fiducial particles in close proximity of each other. The main steps of this algorithm are:
- **1. Fiducial particle detection** - Detection of fiducial particles using the template matching algorithm.
- **2. Cluster detection** - Filtering the set of individual fiducial particles by recognizing clusters of touching fiducial particles and replacing these clusters by their centroid positions.
- **3. Results saving** - Saving the positions of all detected fiducial particles and the positions of fiducial clusters into files.

Load the necessary python libraries:

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import cv2
import math
import numpy as np
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
from skimage import exposure
import matplotlib.pyplot as plt
#from imutils.object_detection import non_max_suppression
from utils_template_matching import non_max_suppression, template_matching, normalize_image, filter_the_template_matching_results
from utils import plot_image, list_to_dataframe, dataframe_to_csv, dataframe_to_xml, dataframe_to_pointcloud, dataframe_to_xml_

Set the path to the input EM image and values for the parameters, load the EM image:

In [ ]:
# Load the imput EM image and template
input_folder = 'E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1'
image_path = Path(os.path.join(input_folder, "240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM.tif"))
print(image_path.exists())

output_folder = Path(os.path.join(input_folder,"output"))
output_folder.mkdir(exist_ok=True)

test_folder = Path('//vironova.com/root/Users/kristinal/Documents/1Test')

image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Parameters for template generation    #Sample1/bin 2 needs size 11, otherwise size 9 works fine
template_size = 9     # Create an empty template of size ('size','size') filled with constant value - 'value' 9, 11
template_value = 80    # 80, 90

# Parameters for template matching
matching_threshold = 0.7  # threshold for template matching
overlap_threshold = 0.1

# Get template dimensions
# h, w = template.shape

# Information about fiducial particles
fiducial_diam = 27     # diameter of fiducial particles in px

## 1. Fiducial particle detection

### Creating a template for the template matching

Fiducial particles have a very distinct central region, characterized by a dark spot surrounded by a lighter gray region. Instead of attempting to match the entire particle using a full template, we focus on detecting the central region only. To accomplish this, the template can be easily generated artificially. We create an initial example template of size (9,9). This example template will be immediately resized to the desired template size which was set up earlier using parameter 'template_size'.

Using a full particle template did not work satisfactory because of the variaty of the region around the particle. 

In [ ]:
# Create the template as an empty template of size ('size','size') filled with constant value - 'value'
example_template = np.full((9, 9), template_value, dtype=np.uint8)

# Define the 3x3 pattern
template_pattern = np.array([[4, 2, 4],
                             [2, 0, 2],
                             [4, 2, 4]])

# Calculate the starting index to place the pattern in the middle of the template
s_idx = (example_template.shape[0] - template_pattern.shape[0]) // 2
e_idx = s_idx + template_pattern.shape[0]
    
# Place the pattern in the middle of the template
example_template[s_idx:e_idx, s_idx:e_idx] = template_pattern

#import bigfish.stack as stack
#template = stack.resize_image(template,(template.shape[0]*2,template.shape[0]*2), method='bilinear')

example_template = example_template.astype(np.uint8)
template = cv2.resize(example_template, (template_size,template_size))

In [ ]:
plot_image(template,(10))
print(template)

Template matching is performed. The matches are first filtered so we ensure that the matches are unique and not overlapping. Then we filter false positives by looking at the 

In [ ]:
# Perform template matching
matching_threshold = 0.7 # 0.68

matches, scores = template_matching(normalize_image(image), template, matching_threshold)
print('Matches:', len(matches), matches)

# Create bounding boxes
w, h = template.shape[::-1]
boxes = [(x, y, x + w, y + h) for (x, y) in matches]    # x,y is the probably the middle of the box, but it does not matter 
                                                        # when calculating the box overlap
# Apply non-maximum suppression for filtering out overlapping boxes
keep_ids = non_max_suppression(np.array(boxes), scores, overlap_threshold)

# Center the matches
loc = [np.asarray(matches[keep_id])+[w//2,h//2] for keep_id in keep_ids]

In [ ]:
# Filter the template matching results based on average intensity
loc1, loc2, loc3 = filter_the_template_matching_results(loc, normalize_image(image), 1, 6, 3) # size of middle square, size of outer square, size of inner square for ring

# Draw rectangles around the matched regions
img2 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
#for pt in loc1:
#    cv2.circle(img2, (pt[0] + w , pt[1] + h ), 1, (0, 128, 0), 2)
      
#for pt in loc2:
#    cv2.circle(img2, (pt[0] - w , pt[1] - h ), 1, (255, 255, 0), 2)

for pt in loc3:
    cv2.rectangle(img2, (pt[0] - fiducial_diam//2 , pt[1] - fiducial_diam//2 ), (pt[0] + fiducial_diam//2 , pt[1] + fiducial_diam//2), (0, 0, 0), 2)
    
# Correct locations are the ones that are in loc3 after filtering
loc = loc3

cv2.imwrite(str(output_folder/'fiducial_detection.png'), img2)
plot_image(img2)

### 2. Cluster detection

Clusters of fiducial particles (FP) will be detected and replaced by 1 point located in the centre of the cluster. Single FP will be removed.

In [ ]:
# Create a black image of the same size as img2, then draw filled circles with the radius of FP at the locations 
# of FP taken from the loc variable, dilate the binary image by circular kernel to connect nearby FP, save the image

img_mask = np.zeros_like(img2)
for pt in loc:
    # draw a filled circle around the fiducial particle
    cv2.circle(img_mask, pt, fiducial_diam//2, (255, 255, 255), -1)
    
# Dilate img_mask by circluar kernel of size 7x7
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
img_mask = cv2.dilate(img_mask, kernel, iterations=1)

plot_image(img_mask)
plt.savefig(str(output_folder/'fiducial_detection_mask.png'))


In [ ]:
# Calculate for each connected component the centroid and the pixel area, if the area is larger then 
# the area of 3 fiducial particles then keep the FP, save the centroid and the area into a list

# Find connected components
_, labels = cv2.connectedComponents(img_mask)

# Calculate the centroid and area of each connected component
centroids = []
areas = []

for label in np.unique(labels):
    if label == 0:             # skip the background
        continue
    
    mask = np.zeros_like(img_mask, dtype=np.uint8)
    mask[labels == label] = 1
    
    moments = cv2.moments(mask)
    #print("Moments: ", moments)

    if moments["m00"] > (3 * math.pi * (fiducial_diam//2)**2) :   # equivalent to the area of 3 fiducial particles
        centroids.append((int(moments["m10"] / moments["m00"]), int(moments["m01"] / moments["m00"])))
        areas.append(int(moments["m00"]))
        #print(moment["m00"])

print("Centroids: ", centroids)
print("Areas: ", areas)

# Draw the centroids on the original image
img3 = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
for centroid in centroids:
    cv2.circle(img3, centroid, 5, (0, 255, 0), -1)

cv2.imwrite(str(output_folder/'fiducial_detection_centroids.png'), img3)
plot_image(img3)

clusters = centroids

### 3. Results saving

Save the FP locations and filtered FP locations into a pandas dataframe with columns: 'id', 'name', 'pos_x', 'pos_y'. Then convert dataframe into numpy array of coordinate pairs (X,Y), XML file and PLY file

In [ ]:
# Save all detected fiducial particles into a Pandas dataframe and export it to different file formats
def save_spots_to_diffent_files (spots, folder, file_name, scale):    
    spots_df = list_to_dataframe(spots, os.path.join(folder, f"{file_name}_df.csv"))                                                     #str(output_folder/"source_all_df.csv")

    dataframe_to_xml(spots_df, os.path.join(folder, f"{file_name}.xml"))                      # import file for ICY ec-CLEM plugin
    dataframe_to_xml(spots_df, os.path.join(folder, f"{file_name}_scaled.xml"), scale)                      # import file for ICY ec-CLEM plugin
    dataframe_to_pointcloud(spots_df, os.path.join(folder, f"{file_name}.ply"))                      # import file for point cloud registration using Probgreg package in Python
    dataframe_to_csv(spots_df, os.path.join(folder, f"{file_name}.csv"))                             # import file for BigWarp ImageJ plugin
    return spots_df

#df = save_spots_to_diffent_files(np.unique((spots_post_decomposition), axis=0), output_folder, "source_all", [scale_y, scale_x])  # y and x are swapped in the dataframe
#print(df)

Save the fiducial particles and detected clusters locations:

In [ ]:
# Save the detected spots 'spots' as representants for regions
scale_x = 1
scale_y = 1
loc_swapped = [(x, y) for y, x in loc]
clusters_swapped = [(x, y) for y, x in clusters]

df = save_spots_to_diffent_files(np.unique((loc_swapped), axis=0), output_folder, "target_all", [scale_y, scale_x])  # y and x are swapped in the dataframe
print(df)

# Save the 'clusters'
df = save_spots_to_diffent_files(np.unique((clusters_swapped), axis=0), output_folder, "target_clusters", [scale_y, scale_x])  # y and x are swapped in the dataframe
print(df) 